In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import os, sys
import pandas as pd
import json
import matplotlib.pyplot as plt

In [3]:
import pandas as pd
import numpy as np
import sqlalchemy as sa
import datetime
from datetime import date
from dateutil.relativedelta import relativedelta
from datetime import timedelta, date

## Main folder

In [4]:
data_folder_path = r"C:\git\CHRIS_SCRIPTS"

*Move to the data folder*

In [5]:
os.chdir(data_folder_path)

*Print the content of the folder, with the linux `ls` command*

## Supporting functions

In [6]:
def query_arc_alchemy(sql):
    # ODBC connection string
    _url ='mssql+pyodbc:///?odbc_connect=DATABASE%3DARCTIMESERIES%3BDRIVER%3D%7BSQL+Server+Native+Client+11.0%7D%3BSERVER%3DARCSQL%3BTrace%3D0%3BTrusted_Connection%3DYes'
    engine =sa.create_engine(_url, echo=False, convert_unicode=True)
    with engine.connect() as connection:
        result =connection.execute(sql)
        data =result.fetchall()
        column_names =result.keys()
        dataframe = pd.DataFrame.from_records(data, columns=column_names)

        return dataframe

In [7]:
def daterange(date1, date2):
    for n in range(int((date2 - date1).days) + 1):
        yield date1 + timedelta(n)


## Data

In [8]:
Costs = pd.read_csv(r'Costs.csv')
Variables = pd.read_csv(r'Variables.csv')

***

In [9]:
import sys, os
sys.path.append(r'C:\dev\code\gmt-amp\python-analytics-platform')
sys.path.append(r'C:\dev\code\gmt-amp\amp-models')

sys.path.append(r"C:\git\CHRIS_SCRIPTS")
from gmt.dna.amp.models.gas.net_back import *
from gmt.dna.amp.models.tests.gas.net_back.test_netback import *

In [10]:
tNB = TestNetBack()

## Transformations

In [11]:
Variables['Days Travel temp'] = Variables['Distances']  \
/ (Costs['Laden - Speed (knots)'].iloc[0] * 24)

In [12]:
tNB.run()
pd.testing.assert_series_equal(tNB.RESULTS.SYM_DAYS_TRAVEL_TEMP,
                              Variables['Days Travel temp'] )

Running: SYM_DAYS_TRAVEL_TEMP
Running: SYM_ADD_DAYS
Running: SYM_DAYS_TRAVEL_ONE_WAY_INC_TRANSIT
Running: SYM_DAYS_TRAVEL_RETURN_INC_TIME_AT_PORT
Running: SYM_BOIL_OFF
Running: SYM_DELIVERED_VOLUME
Running: SYM_REGIONCODE
Done


In [13]:
choices = [0, 1, 1.3]
conditions = [
    (Variables['Canal Fee'] == 'No'),
    (Variables['Canal Fee'] == 'Suez'),
    (Variables['Canal Fee'] == 'Panama')]
Variables['Add days'] = np.select(conditions, choices, default=9999999)

In [14]:
tNB.run()
pd.testing.assert_series_equal(tNB.RESULTS.SYM_ADD_DAYS,
                              Variables['Add days'] )

Running: SYM_DAYS_TRAVEL_TEMP
Running: SYM_ADD_DAYS
Running: SYM_DAYS_TRAVEL_ONE_WAY_INC_TRANSIT
Running: SYM_DAYS_TRAVEL_RETURN_INC_TIME_AT_PORT
Running: SYM_BOIL_OFF
Running: SYM_DELIVERED_VOLUME
Running: SYM_REGIONCODE
Done


In [15]:
Variables['Days Travel (one-way inc. transit)'] = Variables['Days Travel temp'] \
+ Variables['Add days']

In [16]:
tNB.run()
pd.testing.assert_series_equal(tNB.RESULTS.SYM_DAYS_TRAVEL_ONE_WAY_INC_TRANSIT,
                              Variables['Days Travel (one-way inc. transit)'] )

Running: SYM_DAYS_TRAVEL_TEMP
Running: SYM_ADD_DAYS
Running: SYM_DAYS_TRAVEL_ONE_WAY_INC_TRANSIT
Running: SYM_DAYS_TRAVEL_RETURN_INC_TIME_AT_PORT
Running: SYM_BOIL_OFF
Running: SYM_DELIVERED_VOLUME
Running: SYM_REGIONCODE
Done


In [17]:
Variables['Days Travel (Return - inc. time at port)'] = \
                    + Variables['Days Travel (one-way inc. transit)'] * 2 \
                    + Costs['OneWay - Port turnaround time (Load+Discharge) (days)'].iloc[0]

In [18]:
tNB.run()
pd.testing.assert_series_equal(tNB.RESULTS.SYM_DAYS_TRAVEL_RETURN_INC_TIME_AT_PORT,
                              Variables['Days Travel (Return - inc. time at port)'] )

Running: SYM_DAYS_TRAVEL_TEMP
Running: SYM_ADD_DAYS
Running: SYM_DAYS_TRAVEL_ONE_WAY_INC_TRANSIT
Running: SYM_DAYS_TRAVEL_RETURN_INC_TIME_AT_PORT
Running: SYM_BOIL_OFF
Running: SYM_DELIVERED_VOLUME
Running: SYM_REGIONCODE
Done


In [19]:
Variables['Boil-off'] =  \
    Costs['Loaded - Gas cargo (mmbtu)'].iloc[0]  \
    * Costs['Laden - Natural boil-off gas (%/day)'].iloc[0] \
    * Variables[ 'Days Travel (Return - inc. time at port)'] 

In [20]:
tNB.run()
pd.testing.assert_series_equal(tNB.RESULTS.SYM_BOIL_OFF,
                              Variables['Boil-off'] )

Running: SYM_DAYS_TRAVEL_TEMP
Running: SYM_ADD_DAYS
Running: SYM_DAYS_TRAVEL_ONE_WAY_INC_TRANSIT
Running: SYM_DAYS_TRAVEL_RETURN_INC_TIME_AT_PORT
Running: SYM_BOIL_OFF
Running: SYM_DELIVERED_VOLUME
Running: SYM_REGIONCODE
Done


In [21]:
Variables['Delivered Volume'] =   Costs['Loaded - Gas cargo (mmbtu)'].iloc[0]\
                                - Variables['Boil-off']

In [22]:
tNB.run()
pd.testing.assert_series_equal(tNB.RESULTS.SYM_DELIVERED_VOLUME,
                              Variables['Delivered Volume'] )

Running: SYM_DAYS_TRAVEL_TEMP
Running: SYM_ADD_DAYS
Running: SYM_DAYS_TRAVEL_ONE_WAY_INC_TRANSIT
Running: SYM_DAYS_TRAVEL_RETURN_INC_TIME_AT_PORT
Running: SYM_BOIL_OFF
Running: SYM_DELIVERED_VOLUME
Running: SYM_REGIONCODE
Done


In [23]:
choices2 = ['EU', 'EU', 'EU', 'ME', 'ME', 'NEA', 'NEA', 'LatAm', 'EU', 'EU', 'EU', 'EU', 'EU']
conditions2 = [
    (Variables['TerminalTo'] == 'UK (Hook)'),
    
    (Variables['TerminalTo'] == 'UK (Hook)(NBP)'),
    
    (Variables['TerminalTo'] == 'Belgium (Zee)'),
    (Variables['TerminalTo'] == 'India (Dhambol)'),
    (Variables['TerminalTo'] == 'Middle East (UAE)'),
    (Variables['TerminalTo'] == 'China (Guandong)'),
    (Variables['TerminalTo'] == 'Japan (Tokyo)'),
    (Variables['TerminalTo'] == "Argentina (B'Blanca)"),
    (Variables['TerminalTo'] == 'France (Montoir)'),
    (Variables['TerminalTo'] == 'Spain (Barcelona)'),
    (Variables['TerminalTo'] == 'Italy (Rovigo)'),
    
    (Variables['TerminalTo'] == 'Italy (Rovigo)(PSV)'),
    
    (Variables['TerminalTo'] == 'Netherlands (Gate)')]

Variables['RegionCode'] = np.select(conditions2, choices2, default=9999999)

In [24]:
tNB.run()
pd.testing.assert_series_equal(tNB.RESULTS.SYM_REGIONCODE,
                              Variables['RegionCode'] )

Running: SYM_DAYS_TRAVEL_TEMP
Running: SYM_ADD_DAYS
Running: SYM_DAYS_TRAVEL_ONE_WAY_INC_TRANSIT
Running: SYM_DAYS_TRAVEL_RETURN_INC_TIME_AT_PORT
Running: SYM_BOIL_OFF
Running: SYM_DELIVERED_VOLUME
Running: SYM_REGIONCODE
Done


In [25]:
conditions3 = [
    (Variables['TerminalTo'] == "Argentina (B'Blanca)"),
    (Variables['TerminalTo'] == 'Belgium (Zee)'),
    (Variables['TerminalTo'] == 'China (Guandong)'),
    (Variables['TerminalTo'] == 'France (Montoir)'),
    (Variables['TerminalTo'] == 'India (Dhambol)'),
    (Variables['TerminalTo'] == 'Italy (Rovigo)'),
    (Variables['TerminalTo'] == 'Italy (Rovigo)(PSV)'),
    (Variables['TerminalTo'] == 'Japan (Tokyo)'),
    (Variables['TerminalTo'] == 'Middle East (UAE)'),
    (Variables['TerminalTo'] == 'Netherlands (Gate)'),
    (Variables['TerminalTo'] == 'Netherlands (Gate)(TTF)'),
    (Variables['TerminalTo'] == 'Spain (Barcelona)'),
    (Variables['TerminalTo'] == 'Spain (Barcelona)(PVB)'),
    (Variables['TerminalTo'] == 'UK (Hook)'),
    (Variables['TerminalTo'] == 'UK (Hook)(NBP)')]

choices3 = ['LNG_H_DES_Lat_Am_MID_M_USD', 
            'G_H_Zeebrugge-Hub_Pnc', 
            'LNG_JKM_M_USD', 
            'G_H_PEG-Nord_EUR',
            'LNG_DES_ME_W_INDIA_M_USD',
            'LNG_H_DES_Med_MID_M_USD',
            "G_H_PSV_EUR", 
            'LNG_JKM_M_USD', 
            'LNG_DES_ME_W_INDIA_M_USD', 
            'LNG_H_DES_NWE_MID_M_USD',
            'G_H_TTF_95%_EUR',
            'LNG_H_DES_Med_MID_M_USD',
            'G_H_AOC_EUR', 
            'LNG_H_DES_NWE_MID_M_USD',
            'G_H_NBP_95%_USD']
Variables['CurveName'] = np.select(conditions3, choices3, default=9999999)

In [26]:
conditions4 = [        
    (Variables['TerminalFrom'] == 'Sabine'),
    (Variables['TerminalFrom'] == 'Cove point'),
    (Variables['TerminalFrom'] == 'Nigeria'),
    (Variables['TerminalFrom'] == 'Angola'),
    (Variables['TerminalFrom'] == 'Qatar'),
    (Variables['TerminalFrom'] == 'Algeria'),
    (Variables['TerminalFrom'] == 'Yamal'),
    (Variables['TerminalFrom'] == 'Wheatstone'),
    (Variables['TerminalFrom'] == 'Gladstone'),
    (Variables['TerminalFrom'] == 'T&T')]
choices4 = ['G_NYM_HH_M_115%_USD', 
            'G_NYM_HH_M_115%_USD',
            'LNG_JKM_M_0%_USD',
            'LNG_JKM_M_0%_USD',
            'LNG_JKM_M_0%_USD',
            'LNG_JKM_M_0%_USD',
            'LNG_JKM_M_0%_USD',
            'LNG_JKM_M_0%_USD',
            'LNG_JKM_M_0%_USD',
            'LNG_JKM_M_0%_USD']    
Variables['OriginPriceCurve'] = np.select(conditions4, choices4, default=9999999)  

***

## Constants and mappings

In [27]:
weekend = [6, 7]

start_dt = date(2018, 1, 2)
end_dt = date(2020, 11, 26)

In [28]:
Netback = pd.DataFrame()
Netbackwide = pd.DataFrame()

n = len(list(daterange(start_dt, end_dt)))
print(f'There are {n} days to process.\nLeft:', end=' ')

for dt in list(daterange(start_dt, end_dt)):
    print(f'{n-1}', end='; '); n-=1

    if dt.isoweekday() in weekend:
        continue  

    today = dt.day
    if today > 20:
        from_date = dt + pd.DateOffset(months=2)
        to_date = dt + pd.DateOffset(months=2 ,years=3)    
    else:
        from_date = dt + pd.DateOffset(months=1)
        to_date = dt + pd.DateOffset(months=1 ,years=3)


    query_str_1 = f"""
                    SELECT
                      d.[CurveName],
                      datefromparts(year(min(d.PointDate)), month(min(d.PointDate)), 1) as PointDate,
                      AVG(d.[PointValue]) as PointValue
                    FROM [arc_ts_data].[PriceCurveData] d
                    WHERE d.EffectiveDate = '{dt.strftime("%Y-%m-%d")}' and [CurveName] in 
                    ('G_H_NBP.USD', 'G_H_Zeebrugge-Hub.Pnc','LNG_JKM_M.USD', 'G_H_TRS.EUR', 
                    'LNG_DES_ME_W_INDIA_M.USD', 'LNG_H_DES_Med_MID_M.USD', 'LNG_H_DES_JKM_MID_M.USD', 
                    'LNG_H_DES_NWE_MID_M.USD', 'G_H_TTF.EUR', 'G_H_CEGH.EUR', 'G_H_AOC.EUR', 'G_H_PEG-Nord.EUR', 
                    'G_H_PSV.EUR','G_NYM_HH_M.USD' , 'LNG_H_DES_Lat_Am_MID_M.USD', 'FX_curve_cross_EUR.USD', 
                    'FX_curve_input_USD.GBP', 'FREIGHT_H_LNG_M_RolledOver.USD')  
                    and d.PointDate >= '{from_date.strftime("%Y-%m-01")}' and d.PointDate < '{to_date.strftime("%Y-%m-01")}'
                    group by d.CurveId, d.CurveName, d.EffectiveDate, 12 * year(d.PointDate) + month(d.PointDate)
                    order by d.CurveID
                    """
    #------------------------------------------------------Prices---------------------------------------------------------------
    Prices = query_arc_alchemy(query_str_1)    
    
    Prices['CurveName'] = Prices['CurveName'].str.replace('.', '_')    
    Prices['PointDate_Origin'] = pd.to_datetime(Prices['PointDate'])
    Prices['EffectiveDate'] = dt
    Prices['EffectiveDate'] = pd.to_datetime(Prices['EffectiveDate'])
    Prices['PointDate_Destination'] = Prices['PointDate_Origin']    
    
    columns = ['CurveName', 'PointValue', 'PointDate_Origin', 'EffectiveDate','PointDate_Destination']
    Prices = Prices[columns]    
    
    
    
    mapper = Prices.query('CurveName == "FX_curve_cross_EUR_USD"').set_index(['PointDate_Origin'])['PointValue']    
    Prices['FX_curve_cross_EUR_USD'] = Prices['PointDate_Origin'].map(mapper)    
    Prices['FX_curve_cross_EUR_USD'] = Prices['FX_curve_cross_EUR_USD'].fillna(method='ffill')    
    
    Prices['PointValue'] = np.where(Prices['CurveName'].str.contains("_EUR", case=False),
                                    Prices['PointValue'].apply(pd.to_numeric, downcast='float') / Prices['FX_curve_cross_EUR_USD'].apply(pd.to_numeric, downcast='float') / 3.412 * 0.95,
                                    Prices['PointValue'])
    
    mapper = Prices.query('CurveName == "FX_curve_input_USD_GBP"').set_index(['PointDate_Origin'])['PointValue']
    Prices['FX_curve_input_USD_GBP'] = Prices['PointDate_Origin'].map(mapper)    
    Prices['FX_curve_input_USD_GBP'] = Prices['FX_curve_input_USD_GBP'].fillna(method='ffill')
    
        
    Prices['PointValue'] = np.where(Prices['CurveName'].str.contains("_Pnc", case=False), 
                                    Prices['PointValue'].apply(pd.to_numeric, downcast='float') / Prices['FX_curve_input_USD_GBP'].apply(pd.to_numeric, downcast='float') / 10, 
                                    Prices['PointValue'])
    
    
    Prices['PointValue'] = np.where(Prices['CurveName'].str.contains("G_NYM_HH_M_USD", case=False), 
                                    Prices['PointValue'].apply(pd.to_numeric, downcast='float') * 1.15, 
                                    Prices['PointValue'])
    
    Prices['CurveName'] = np.where(Prices['CurveName'] == 'G_NYM_HH_M_USD',
                                   'G_NYM_HH_M_115%_USD',
                                   Prices['CurveName'])

    
    #INCREASES THE NBP PRICE BY 15% IN-LINE WITH CHENIERE CONTRACTS
    Prices['PointValue'] = np.where(Prices['CurveName'].str.contains("G_H_NBP_USD", case=False), 
                                    Prices['PointValue'].apply(pd.to_numeric, downcast='float') * 0.95, 
                                    Prices['PointValue'])
        
    Prices['CurveName'] = np.where((Prices['CurveName'] == 'G_H_NBP_USD'),
                                   'G_H_NBP_95%_USD',
                                   Prices['CurveName'])
    
    
    

    #INCREASES THE NBP PRICE BY 15% IN-LINE WITH CHENIERE CONTRACTS
    Prices['PointValue'] = np.where(Prices['CurveName'].str.contains("G_H_TTF_EUR", case=False), 
                                    Prices['PointValue'].apply(pd.to_numeric, downcast='float') * 0.95, 
                                    Prices['PointValue'])
    
    Prices['CurveName'] = np.where(
                                    (Prices['CurveName'] == 'G_H_TTF_EUR'),
                                    'G_H_TTF_95%_EUR',
                                    Prices['CurveName']
    )

    
    
    #DECREASES THE JKM PRICE BY 100% 
    Prices['PointValue'] = np.where(Prices['CurveName'].str.contains("G_H_CEGH_EUR", case=False), 
                                    Prices['PointValue'].apply(pd.to_numeric, downcast='float') * 0, 
                                    Prices['PointValue'])
    
    Prices['CurveName'] = np.where(Prices['CurveName'] == 'G_H_CEGH_EUR',
                                   'LNG_JKM_M_0%_USD',
                                   Prices['CurveName'])


    


    #------------------------------------------STOPPED-----------------------------------------------------------------------
    
    #---------------------------------------------------------------------------------------------------------------------------
    #------------------------------------------------------Variables - the same as Variables ------------------------------
    #---------------------------------------------------------------------------------------------------------------------------
    date_rng = pd.date_range(start=dt, freq='MS', periods=12, name='Dates')
    combining_date = []
    
    for d in date_rng:
        Variables['PointDate_Origin'] = d
        combining_date.append(Variables.copy())       

    Variables = pd.concat(combining_date)    
    mapper = Prices.query('CurveName == "FREIGHT_H_LNG_M_RolledOver_USD"').set_index(['PointDate_Origin'])['PointValue']
    Variables['Shipping Cost'] = np.where(Variables['Days Travel (Return - inc. time at port)'] > 30, 
                  ((Variables['PointDate_Origin'] + pd.DateOffset(months=1)).map(mapper) + Variables['PointDate_Origin'].map(mapper)) / 2, 
                  Variables['PointDate_Origin'].map(mapper))        
    Variables[['Regas Costs', 'Port Charges', 'Boil-off', 'Delivered Volume', 'Shipping Cost']] = Variables[
        ['Regas Costs', 'Port Charges', 'Boil-off', 'Delivered Volume', 'Shipping Cost']].apply(pd.to_numeric,
                                                                                                downcast='float')
    Variables['ShippingCostOutput'] = (
        Variables['Shipping Cost'] *
        Variables['Days Travel (Return - inc. time at port)'] /
        Costs['Loaded - Gas cargo (mmbtu)'].iloc[0]
    )

    # Maps Price Curve to a Terminal Location so that Boil off can be calculated

    #---------------------------------------------------------------------------------------------------------------------------
    #------------------------------------------------------NetbackwideOneDt-----------------------------------------------------
    #---------------------------------------------------------------------------------------------------------------------------
    
    NetbackwideOneDt = Variables.merge(
        Prices[['PointDate_Origin','CurveName','EffectiveDate']], how='inner', on=['PointDate_Origin', 'CurveName'])
    
    
    
    
    NetbackwideOneDt = NetbackwideOneDt.merge(Prices[['PointDate_Origin', 'CurveName','PointValue']], how='inner', right_on=['PointDate_Origin', 'CurveName'],
                                   left_on=['PointDate_Origin', 'OriginPriceCurve'])    
    NetbackwideOneDt = NetbackwideOneDt.drop(columns=['CurveName_y'])   
    NetbackwideOneDt.rename(columns={'CurveName_x':'CurveName_Destination','PointValue':'PointValue_Origin'}, inplace=True)     
    NetbackwideOneDt['PointDate_Destination'] = NetbackwideOneDt['PointDate_Origin']    
    NetbackwideOneDt['PointDate_Destination'] = np.where(NetbackwideOneDt['Days Travel (Return - inc. time at port)'] > 30, 
                   (NetbackwideOneDt['PointDate_Destination'] + pd.DateOffset(months=1)), 
                   NetbackwideOneDt['PointDate_Destination'])    
    NetbackwideOneDt = NetbackwideOneDt.merge(Prices[['PointDate_Destination','CurveName','PointValue']], right_on=['PointDate_Destination', 'CurveName'],
                                   left_on=['PointDate_Destination', 'CurveName_Destination'])
    NetbackwideOneDt.rename(columns={'PointValue':'PointValue_Destination'}, inplace=True)
    NetbackwideOneDt = NetbackwideOneDt.drop(columns=['CurveName'])
    NetbackwideOneDt[['Boil-off', 'PointValue_Destination','PointValue_Origin']] = NetbackwideOneDt[['Boil-off', 'PointValue_Destination', 'PointValue_Origin']].apply(pd.to_numeric,downcast='float')
    NetbackwideOneDt['BoilOffOutput'] = (
            NetbackwideOneDt['PointValue_Destination'] * NetbackwideOneDt['Boil-off'] / Costs['Loaded - Gas cargo (mmbtu)'].iloc[0])
    NetbackwideOneDt['FuelOutput'] = np.where(NetbackwideOneDt['TerminalFrom'] == 'Yamal', \
                                              ((Costs['OneWay - Port turnaround time (Load+Discharge) (days)'].iloc[0] *
                                                Costs['Fuel consumption in port (tonnes/day)'].iloc[0] *
                                                Costs['USD/tonne - HFO'].iloc[0]) + \
                                               (Costs['USD/tonne - HFO'].iloc[0] *
                                                Costs['Fuel oil consumption (tonnes/day)'].iloc[0] * 2 * NetbackwideOneDt[
                                                    'Days Travel (one-way inc. transit)'])) /
                                              Costs['Loaded - Gas cargo (mmbtu)'].iloc[0], 0.06)
    NetbackwideOneDt['CanalCostsOutput'] = np.where(NetbackwideOneDt['Canal Fee'] == 'Panama', (
            Costs['Panama - Total ($)'].iloc[0] / Costs['Loaded - Gas cargo (mmbtu)'].iloc[0]), \
                                                    np.where(NetbackwideOneDt['Canal Fee'] == 'Suez', (
                                                            Costs['Suez - Total ($)'].iloc[0] /
                                                            Costs['Loaded - Gas cargo (mmbtu)'].iloc[0]), 0))
    NetbackwideOneDt['PortCostsOutput'] = (NetbackwideOneDt['Port Charges'] / Costs['Loaded - Gas cargo (mmbtu)'].iloc[0])
    NetbackwideOneDt['TotalCostIncluding(Regas&Port)'] = NetbackwideOneDt['ShippingCostOutput'] + \
                                                         NetbackwideOneDt['BoilOffOutput'] + \
                                                         NetbackwideOneDt['FuelOutput'] + \
                                                         NetbackwideOneDt['CanalCostsOutput'] + \
                                                         NetbackwideOneDt['PortCostsOutput'] + \
                                                         NetbackwideOneDt['Regas Costs']
    NetbackwideOneDt['TotalCostIncluding(Regas)'] = NetbackwideOneDt['ShippingCostOutput'] + \
                                                    NetbackwideOneDt['BoilOffOutput'] + \
                                                    NetbackwideOneDt['FuelOutput'] + \
                                                    NetbackwideOneDt['CanalCostsOutput'] + \
                                                    NetbackwideOneDt['Regas Costs']
    NetbackwideOneDt['TotalCostIncluding(Port)'] = NetbackwideOneDt['ShippingCostOutput'] + \
                                                   NetbackwideOneDt['BoilOffOutput'] + \
                                                   NetbackwideOneDt['FuelOutput'] + \
                                                   NetbackwideOneDt['CanalCostsOutput'] + \
                                                   NetbackwideOneDt['PortCostsOutput']
    NetbackwideOneDt['TotalCost'] = NetbackwideOneDt['ShippingCostOutput'] + \
                                    NetbackwideOneDt['BoilOffOutput'] + \
                                    NetbackwideOneDt['FuelOutput'] + \
                                    NetbackwideOneDt['CanalCostsOutput']
    NetbackwideOneDt['TotalCost(Sunk Shiping)'] = NetbackwideOneDt['ShippingCostOutput'] + \
                                NetbackwideOneDt['BoilOffOutput'] + \
                                NetbackwideOneDt['FuelOutput'] + \
                                NetbackwideOneDt['CanalCostsOutput']    

    NetbackwideOneDt['Variable_Shipping_Roundtrip'] = NetbackwideOneDt['PointValue_Destination'] - NetbackwideOneDt['BoilOffOutput'] -  NetbackwideOneDt['FuelOutput'] - NetbackwideOneDt['PortCostsOutput'] - NetbackwideOneDt['PointValue_Origin']
    NetbackwideOneDt['Variable_Shipping_Single'] = NetbackwideOneDt['PointValue_Destination'] - 0.5*NetbackwideOneDt['BoilOffOutput'] -  NetbackwideOneDt['FuelOutput'] - NetbackwideOneDt['PortCostsOutput'] - NetbackwideOneDt['PointValue_Origin']
    NetbackwideOneDt['Full_Shipping_Rountrip'] = NetbackwideOneDt['PointValue_Destination'] - NetbackwideOneDt['TotalCostIncluding(Port)'] - NetbackwideOneDt['PointValue_Origin']


    #---------------------------------------------------------------------------------------------------------------------------
    #------------------------------------------------------NetbackOneDt--------------------------------------------------------------
    #---------------------------------------------------------------------------------------------------------------------------
    mask = ['EffectiveDate', 'PointDate_Origin', 'PointDate_Destination',
                                'RegionFrom', 'TerminalFrom', 'TerminalTo', 'TotalCost', 'TotalCostIncluding(Regas&Port)',
                                'TotalCostIncluding(Regas)',
                                'TotalCostIncluding(Port)','Full_Shipping_Rountrip','Variable_Shipping_Single','Variable_Shipping_Roundtrip']
    NetbackOneDt = NetbackwideOneDt[mask]

    Netback = Netback.append(NetbackOneDt, ignore_index = True )
#     Netback.groupby(['TerminalFrom','TerminalTo'])

    Netbackwide = Netbackwide.append(NetbackwideOneDt, ignore_index = True )
#     NetbackwideOneDt2.groupby(['TerminalFrom','TerminalTo'])

There are 1060 days to process.
Left: 1059; 1058; 1057; 1056; 1055; 1054; 1053; 

KeyboardInterrupt: 

In [ ]:
Netback.shape

In [ ]:
Netbackwide.shape

## Store results

In [ ]:
Netback.to_csv('netback.csv', index=False)
Netbackwide.to_csv('netback_wide.csv', index=False)

In [ ]:
Netback.to_csv(r'C:\temp\new_vars.csv', index=False)

## Modifications